In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy import stats
from scipy import spatial
import time
import math
from IPython.display import display
from collections import Counter

np.random.seed(777)
us = m21.environment.UserSettings()

# us['musescoreDirectPNGPath']="/home/sirivasv/Downloads/MuseScore-3.4.2-x86_64.AppImage"
us['musescoreDirectPNGPath']='/home/sirivasv/.local/bin/MuseScore-3.5.2.312125617-x86_64.AppImage'

## Functions

In [2]:
DIV_CONST = 4

In [3]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [4]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [5]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
            
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [6]:
def get_SCLM_v100(melody_w_times_A, melody_w_times_B):
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    # Actual DP implementation
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            # If there is a match a possible solution is the previous plus one
            curr_value = 0
            
            tot_delta_time = (float(melody_w_times_A[i][1]) + float(melody_w_times_B[j][1])) / float(DIV_CONST)
            tot_diff_time = np.abs(float(melody_w_times_A[i][1]) - float(melody_w_times_B[j][1]))
            
            
            if (melody_w_times_A[i][3] == melody_w_times_B[j][3]) and (tot_diff_time <= tot_delta_time):
                curr_value = memo[i + 1][j + 1] + 1
                
            # The actual solution is the maximum between the one if there is a match, or skip on the melody A or melody B
            curr_value = max(curr_value, max(memo[i + 1][j], memo[i][j + 1]))
            
            # Save the solution
            memo[i][j] = curr_value
    
    # With the memoization table we can retrieve the actual melody
    i = 0
    j = 0
    SCLM = []
    while i != lim_A and j != lim_B:
    
        if ((memo[i + 1][j + 1] + 1) == memo[i][j]):
            SCLM.append((i, j))
            i += 1
            j += 1
        elif (memo[i + 1][j] == memo[i][j]):
            i += 1
        elif (memo[i][j + 1] == memo[i][j]):
            j += 1
    
    return SCLM

In [7]:
def get_max_timestamp_dif(melody_w_times_A, melody_w_times_B):
    return max(
        melody_w_times_A[len(melody_w_times_A) - 1][0] - melody_w_times_A[0][0],
        melody_w_times_B[len(melody_w_times_B) - 1][0] - melody_w_times_B[0][0]
    )

In [8]:
def getDifSCLM(melody_w_times_A, melody_w_times_B, sclm):
    
    # If there is no sclm or it is just one return max possible value
    if (len(sclm) <= 1):
        return get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    
    
    # Initialize the arrays
    T_A = np.zeros(shape=(len(sclm) - 1))
    T_B = np.zeros(shape=(len(sclm) - 1))
    T_C = np.zeros(shape=(len(sclm) - 1))
    Dif_ = np.zeros(shape=(len(sclm) - 1))
    
    for i in range(1, len(sclm)):
        T_A[i - 1] = melody_w_times_A[sclm[i][0]][0] - melody_w_times_A[sclm[i-1][0]][0]
        T_B[i - 1] = melody_w_times_B[sclm[i][1]][0] - melody_w_times_B[sclm[i-1][1]][0]
        T_C[i - 1] = np.abs(T_A[i - 1] - T_B[i - 1])
    
    T_C_mean = np.mean(T_C)
    
    for i in range(0, len(T_B)):
        T_B[i] += T_C_mean
        Dif_[i] = T_A[i] - T_B[i]
    
    return T_C_mean
    

In [9]:
def get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B):
    
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        return 1
    
    # Initialize result variable
    result_value = 0
    
    # Get Keys
    key_A = getSongKeyFromMelody_W_Times(melody_w_times_A)
    key_B = getSongKeyFromMelody_W_Times(melody_w_times_B)
    
    # D1: Scale  
    scale_dif1 = 0
    if (key_A.name != key_B.name):
        scale_dif1 = W1
    result_value += scale_dif1
    
    # D2: Mode  
    mode_dif2 = 0
    if (key_A.mode != key_B.mode):
        mode_dif2 = W2
    result_value += mode_dif2
    
    # Get SCLM v100
    sclm = get_SCLM_v100(melody_w_times_A, melody_w_times_B)
    
    # Get max len
    max_len = max(len(melody_w_times_A), len(melody_w_times_B))
    
    # D3: SCLM Length
    sclmlen_dif3 = ((max_len - len(sclm)) / max_len) * W3
    result_value += sclmlen_dif3
    
    # Get the Diff on temporal spacing in the SCLM
    dif_sclm = getDifSCLM(melody_w_times_A, melody_w_times_B, sclm)
    
    # D4: dif in sclm
    max_timestamp_dif = get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    sclmdif_dif4 = (dif_sclm / max_timestamp_dif) * W4
    result_value += sclmdif_dif4
    
    return result_value

In [10]:
W1 = 0.0
W2 = 0.0
W3 = 1.0
W4 = 0.0

In [11]:
def plot_similarities(similarities):
    fig, axs_to_plot = plt.subplots(1, 1, figsize=(12, 6))
    rects = axs_to_plot.bar(list(map(str, [i for i in range(1,len(similarities) + 1)])), similarities, color="green")
    axs_to_plot.set_title(label="Similarities of generated solos relative to corpus")

### Local Aligment similarity

In [12]:
def pitch_rater(element_A, element_B):
    if element_A == element_B:
        return 1.0
    else:
        return -1.0

In [13]:
def get_Local_Alignment_from_melody_w_times(melody_w_times_A, melody_w_times_B, insertion_weight=-.5, deletion_weight=-.5, sim_score=pitch_rater):
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        return 1.0
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    max_score = 0.0
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            from_left = memo[i][j + 1] + deletion_weight
            from_top = memo[i + 1][j] + insertion_weight
            diag = memo[i + 1][j + 1]  + sim_score(melody_w_times_A[i][2],melody_w_times_B[j][2])
            
            
            memo[i][j] = max(from_top, from_left, diag, 0.0)
            if memo[i][j] > max_score:
                max_score = memo[i][j]
    return -max_score

### Euclidean Distance

In [14]:
def get_pitch_vector(melody_w_times):
    return [float(x[2]) for x in melody_w_times]

In [15]:
def get_Ecuclidean_Distance_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    pitch_vector_query = get_pitch_vector(melody_w_times_query)
    pitch_vector_test = get_pitch_vector(melody_w_times_test)
    
    max_len = max(len(pitch_vector_query), len(pitch_vector_test))
    # max_len = float(max(len(pitch_vector_query), len(pitch_vector_test)))
    # min_len = min(len(pitch_vector_query), len(pitch_vector_test))
    
    pitch_vector_query.extend([0] * (max_len - len(pitch_vector_query)))
    pitch_vector_test.extend([0] * (max_len - len(pitch_vector_test)))
    
    sim = spatial.distance.euclidean(pitch_vector_query, pitch_vector_test)
    
    return sim / len(pitch_vector_query)
    # return sim / max_len

### SIAM

In [16]:
def get_SIAM_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    start_onset = melody_w_times_query[0][0]
    query_points = [(s[0] - start_onset, s[2]) for s in melody_w_times_query]
    
    translation_vectors = []
    translation_vectors_with_position = []
    
    test_points = np.array([(s[0], s[2]) for s in melody_w_times_test])
    
    for p in query_points:
        vectors = (test_points - p)
        translation_vectors.extend([tuple(v) for v in vectors])
        translation_vectors_with_position.append((p[0], [tuple(v) for v in vectors]))
    
    grouped_vectors = dict(Counter(translation_vectors))
    similarity = max([grouped_vectors[k] for k in grouped_vectors])     
    
    return -(similarity / float(len(query_points)))

### Control for similarity measure to use

In [17]:
def similarity_evaluation(measure_key, melody_w_times_A, melody_w_times_B):
    
    if measure_key == "MTRC_v100":
        return get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "LA":
        return get_Local_Alignment_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "ED":
        return get_Ecuclidean_Distance_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "SIAM":
        return get_SIAM_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    
    return 1.0

# Groups for analysis
 * Reference for the axis
 * Groups of dataset

In [18]:
Y_AXIS_MIDI_PATHS = [
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/CORPUS/MIDI/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_SOLO_LEN12/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/MilesDavis/MIDI_Grammar_SOLO_LEN12/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CharlieParker/MIDI_Grammar_SOLO_LEN12/"
]

In [19]:
X_AXIS_MIDI_PATHS = [
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/CORPUS/MIDI/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_SOLO_LEN12/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/MilesDavis/MIDI_Grammar_SOLO_LEN12/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CharlieParker/MIDI_Grammar_SOLO_LEN12/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_TRADE_Caleb/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_TRADE_Miles/",
    "/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_TRADE_CharlieParker/"
]

In [20]:
# SIMILARITY_MEASURE_KEY = "MTRC_v100"
# SIMILARITY_MEASURE_KEY = "LA"
# SIMILARITY_MEASURE_KEY = "ED"
SIMILARITY_MEASURE_KEY = "SIAM"

In [21]:
all_similarities_store = {}
for MIDI_path_query in Y_AXIS_MIDI_PATHS:
    for MIDI_path_test in X_AXIS_MIDI_PATHS:

        similarities_from_references = []
        similarities_all_v_all = {}
        for root_ref, dirs_ref, files_ref in os.walk(MIDI_path_query):
            for name_ref in files_ref:
                # print("+++++++++++++++++++++++++++++")
                # print(name_ref)
                melody_score_A = m21.converter.parseFile(os.path.join(root_ref, name_ref))
                midi_tracks_A = m21.midi.translate.streamToMidiFile(melody_score_A)
                melody_w_times_A = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)

                similarities_from_reference = []
                similarities_all_v_all[name_ref] = {}
                for root, dirs, files in os.walk(MIDI_path_test):
                    for name in files:

                        # print(name)
                        melody_score_B = m21.converter.parseFile(os.path.join(root, name))
                        midi_tracks_B = m21.midi.translate.streamToMidiFile(melody_score_B)
                        melody_w_times_B = getMelodyDeltaTimes(midi_tracks_B.tracks[0].events)
                        
                        if (name == name_ref) and (root == root_ref):
                            current_similarity = 0.0;
                        else:
                            current_similarity = similarity_evaluation(
                                SIMILARITY_MEASURE_KEY, melody_w_times_A, melody_w_times_B)

                        similarities_from_reference.append(current_similarity)
                        similarities_all_v_all[name_ref][name] = current_similarity
                        # print(current_similarity)
                similarities_from_references.append(similarities_from_reference)
        similarities_df = pd.DataFrame(similarities_all_v_all)
        similarities_df = similarities_df.reindex(sorted(similarities_df.columns), axis=1)
        similarities_df = similarities_df.sort_index()
        similarities_df = similarities_df.replace(0, np.NaN)
        display(similarities_df)
        print(similarities_df.mean(axis=1))
        keyname_sim = MIDI_path_query+":"+MIDI_path_test
        print(keyname_sim)
        all_similarities_store[keyname_sim] = similarities_df.to_dict('index')

,rola1.mid,rola2.mid,rola3.mid,rola4.mid,rola5.mid
rola1.mid,NaN,-0.178571,-0.146341,-0.145455,-0.115385
rola2.mid,-0.084746,NaN,-0.121951,-0.090909,-0.096154
rola3.mid,-0.101695,-0.178571,NaN,-0.109091,-0.134615
rola4.mid,-0.135593,-0.178571,-0.146341,NaN,-0.096154
rola5.mid,-0.101695,-0.178571,-0.170732,-0.090909,NaN


rola1.mid   -0.146438
rola2.mid   -0.098440
rola3.mid   -0.130993
rola4.mid   -0.139165
rola5.mid   -0.135477
dtype: float64
/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/CORPUS/MIDI/:/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/CORPUS/MIDI/


,rola1.mid,rola2.mid,rola3.mid,rola4.mid,rola5.mid
CalebRascon_Grammar_SOLO_LEN12_001.mid,-0.050847,-0.107143,-0.121951,-0.072727,-0.115385
CalebRascon_Grammar_SOLO_LEN12_002.mid,-0.067797,-0.142857,-0.097561,-0.072727,-0.096154
CalebRascon_Grammar_SOLO_LEN12_003.mid,-0.067797,-0.142857,-0.097561,-0.072727,-0.096154
CalebRascon_Grammar_SOLO_LEN12_004.mid,-0.050847,-0.107143,-0.121951,-0.054545,-0.076923
CalebRascon_Grammar_SOLO_LEN12_005.mid,-0.050847,-0.071429,-0.073171,-0.054545,-0.076923


CalebRascon_Grammar_SOLO_LEN12_001.mid   -0.093611
CalebRascon_Grammar_SOLO_LEN12_002.mid   -0.095419
CalebRascon_Grammar_SOLO_LEN12_003.mid   -0.095419
CalebRascon_Grammar_SOLO_LEN12_004.mid   -0.082282
CalebRascon_Grammar_SOLO_LEN12_005.mid   -0.065383
dtype: float64
/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/CORPUS/MIDI/:/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_SOLO_LEN12/


,rola1.mid,rola2.mid,rola3.mid,rola4.mid,rola5.mid
MilesDavis_Grammar_SOLO_LEN12_001.mid,-0.067797,-0.178571,-0.097561,-0.072727,-0.115385
MilesDavis_Grammar_SOLO_LEN12_002.mid,-0.067797,-0.142857,-0.097561,-0.072727,-0.115385
MilesDavis_Grammar_SOLO_LEN12_003.mid,-0.067797,-0.107143,-0.121951,-0.090909,-0.096154
MilesDavis_Grammar_SOLO_LEN12_004.mid,-0.050847,-0.107143,-0.097561,-0.072727,-0.096154
MilesDavis_Grammar_SOLO_LEN12_005.mid,-0.067797,-0.107143,-0.121951,-0.072727,-0.096154


...

trade_Caleb_Caleb_rola1.mid   -0.074335
trade_Caleb_Caleb_rola2.mid   -0.063057
trade_Caleb_Caleb_rola3.mid   -0.049874
trade_Caleb_Caleb_rola4.mid   -0.049874
trade_Caleb_Caleb_rola5.mid   -0.048865
dtype: float64
/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CharlieParker/MIDI_Grammar_SOLO_LEN12/:/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_TRADE_Caleb/


,CharlieParker_Grammar_SOLO_LEN12_001.mid,CharlieParker_Grammar_SOLO_LEN12_002.mid,CharlieParker_Grammar_SOLO_LEN12_003.mid,CharlieParker_Grammar_SOLO_LEN12_004.mid,CharlieParker_Grammar_SOLO_LEN12_005.mid
trade_Caleb_Miles_rola1.mid,-0.047619,-0.047619,-0.043478,-0.067797,-0.057143
trade_Caleb_Miles_rola2.mid,-0.059524,-0.059524,-0.054348,-0.084746,-0.071429
trade_Caleb_Miles_rola3.mid,-0.035714,-0.035714,-0.032609,-0.067797,-0.057143
trade_Caleb_Miles_rola4.mid,-0.047619,-0.047619,-0.043478,-0.101695,-0.057143
trade_Caleb_Miles_rola5.mid,-0.059524,-0.071429,-0.065217,-0.067797,-0.071429


trade_Caleb_Miles_rola1.mid   -0.052731
trade_Caleb_Miles_rola2.mid   -0.065914
trade_Caleb_Miles_rola3.mid   -0.045795
trade_Caleb_Miles_rola4.mid   -0.059511
trade_Caleb_Miles_rola5.mid   -0.067079
dtype: float64
/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CharlieParker/MIDI_Grammar_SOLO_LEN12/:/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_TRADE_Miles/


,CharlieParker_Grammar_SOLO_LEN12_001.mid,CharlieParker_Grammar_SOLO_LEN12_002.mid,CharlieParker_Grammar_SOLO_LEN12_003.mid,CharlieParker_Grammar_SOLO_LEN12_004.mid,CharlieParker_Grammar_SOLO_LEN12_005.mid
trade_Caleb_CharlieParker_rola1.mid,-0.071429,-0.059524,-0.065217,-0.084746,-0.071429
trade_Caleb_CharlieParker_rola2.mid,-0.071429,-0.059524,-0.043478,-0.101695,-0.071429
trade_Caleb_CharlieParker_rola3.mid,-0.059524,-0.095238,-0.054348,-0.118644,-0.100000
trade_Caleb_CharlieParker_rola4.mid,-0.071429,-0.083333,-0.086957,-0.118644,-0.085714
trade_Caleb_CharlieParker_rola5.mid,-0.071429,-0.059524,-0.054348,-0.067797,-0.057143


trade_Caleb_CharlieParker_rola1.mid   -0.070469
trade_Caleb_CharlieParker_rola2.mid   -0.069511
trade_Caleb_CharlieParker_rola3.mid   -0.085551
trade_Caleb_CharlieParker_rola4.mid   -0.089215
trade_Caleb_CharlieParker_rola5.mid   -0.062048
dtype: float64
/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CharlieParker/MIDI_Grammar_SOLO_LEN12/:/home/sirivasv/Documents/MCC/MCC-Thesis/CCEvaluation/CalebRascon/MIDI_Grammar_TRADE_CharlieParker/


In [22]:
len(list(all_similarities_store.keys()))

28

In [23]:
with open('./CCEvaluation_groups_SIAM.json', 'w') as outfile:
    json.dump(all_similarities_store, outfile)